In [1]:
import os
from googleapiclient.discovery import build

GOOGLE_CSE_ID = os.environ.get("GOOGLE_CSE_ID")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")


def get_metadata_result(result):
    metadata_result = {
        "title": result["title"],
        "link": result["link"],
    }
    if "snippet" in result:
        metadata_result["snippet"] = result["snippet"]
    return metadata_result


def get_google_search(query):
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    request = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=3)
    response = request.execute()
    results = response.get("items", [])
    if not results:
        return [{"Result": "No good Google Search Result was found"}]
    metadata_results = [get_metadata_result(result) for result in results]
    return metadata_results

In [2]:
import autogen

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")
llm_config = {
    "config_list": config_list,
    "request_timeout": 120,
    "temperature": 0,
    "functions": [
        {
            "name": "get_google_search",
            "description": "Get Google Search Results",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search Query"}
                },
                "required": ["query"],
            },
        }
    ],
}

In [3]:
assistant = autogen.AssistantAgent(name="assitant", llm_config=llm_config)
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NERVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "coding"},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"get_google_search": get_google_search},
)

In [4]:
user_proxy.initiate_chat(
    assistant,
    message="""
Search google news in the topic of "COVID-19", and write a news based on the search results.
You should avoid bullet points. Instead, use more expressive words to describe the form the news that's like written by a well regonized journalist.""",
)

user_proxy (to assitant):


Search google news in the topic of "COVID-19", and write a news based on the search results.
You should avoid bullet points. Instead, use more expressive words to describe the form the news that's like written by a well regonized journalist.

--------------------------------------------------------------------------------
assitant (to user_proxy):

***** Suggested function Call: get_google_search *****
Arguments: 
{
  "query": "COVID-19"
}
******************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION get_google_search...
user_proxy (to assitant):

***** Response from calling function "get_google_search" *****
[{'title': 'Coronavirus Disease 2019 (COVID-19) | CDC', 'link': 'https://www.cdc.gov/coronavirus/2019-ncov/index.html', 'snippet': 'Jul 13, 2023 ... Highlights · Symptoms of COVID-19. People may experience sudden fever or